In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Ceará - Valor Cimento Portland,Distrito Federal - IDH,Distrito Federal - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - Desemprego,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Consumo de Cimento (t)
0,2003,3.580301,0.826971,2006.510747,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.293882,1.184017e+08,3.570616e+06,43.639430,1.092465e+08,635.841000
1,2004,3.521521,0.828078,2207.532012,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.221939,1.201946e+08,3.601434e+06,43.781872,1.099185e+08,771.216000
2,2005,3.627457,0.829060,2364.080883,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.194578,1.218307e+08,3.601501e+06,43.900038,1.104515e+08,855.531000
3,2006,3.768483,0.829921,2552.261933,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.106637,1.233297e+08,3.585655e+06,44.002809,1.109025e+08,747.184000
4,2007,3.869648,0.830662,2607.657436,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.077713,1.246449e+08,3.601930e+06,44.069902,1.112206e+08,808.342000
5,2008,3.929539,0.831284,2685.923746,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.042711,1.257796e+08,3.618753e+06,44.111562,1.114321e+08,909.713000
6,2009,4.081058,0.831790,2935.357560,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.070885,1.266042e+08,3.629545e+06,44.141542,1.114727e+08,991.263000
7,2010,4.272214,0.832181,3227.460076,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.035310,1.296005e+08,3.625996e+06,44.116626,1.120321e+08,1167.604000
8,2011,4.332790,0.832459,3532.866309,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.024653,1.323025e+08,3.587906e+06,44.022754,1.124103e+08,1277.890000
9,2012,4.383851,0.832380,3799.093892,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,10.763245,1.348108e+08,3.550906e+06,43.926397,1.127652e+08,1261.667000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Ceará - Valor Cimento Portland,Distrito Federal - IDH,Distrito Federal - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - Desemprego,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado
0,3.580301,0.826971,2006.510747,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.293882,1.184017e+08,3.570616e+06,43.639430,1.092465e+08
1,3.521521,0.828078,2207.532012,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.221939,1.201946e+08,3.601434e+06,43.781872,1.099185e+08
2,3.627457,0.829060,2364.080883,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.194578,1.218307e+08,3.601501e+06,43.900038,1.104515e+08
3,3.768483,0.829921,2552.261933,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.106637,1.233297e+08,3.585655e+06,44.002809,1.109025e+08
4,3.869648,0.830662,2607.657436,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.077713,1.246449e+08,3.601930e+06,44.069902,1.112206e+08
5,3.929539,0.831284,2685.923746,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.042711,1.257796e+08,3.618753e+06,44.111562,1.114321e+08
6,4.081058,0.831790,2935.357560,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.070885,1.266042e+08,3.629545e+06,44.141542,1.114727e+08
7,4.272214,0.832181,3227.460076,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.035310,1.296005e+08,3.625996e+06,44.116626,1.120321e+08
8,4.332790,0.832459,3532.866309,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.024653,1.323025e+08,3.587906e+06,44.022754,1.124103e+08
9,4.383851,0.832380,3799.093892,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,10.763245,1.348108e+08,3.550906e+06,43.926397,1.127652e+08


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      771.216000
1      855.531000
2      747.184000
3      808.342000
4      909.713000
5      991.263000
6     1167.604000
7     1277.890000
8     1261.667000
9     1152.069000
10     979.998857
11     811.194686
12     638.818457
13     584.646000
14     571.130000
15     637.028000
16     662.625000
17     789.947000
18     786.865000
19            NaN
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Ceará - Valor Cimento Portland,Distrito Federal - IDH,Distrito Federal - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - Desemprego,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado
0,-1.514981,-2.345479,-1.736587,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.834235,-1.632321,0.417571,-1.906699,-2.068818
1,-1.676914,-1.769333,-1.350490,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.872827,-1.412926,0.838655,-0.904987,-1.587746
2,-1.385074,-1.257671,-1.049810,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.887504,-1.212721,0.839563,-0.074001,-1.206231
3,-0.996563,-0.809538,-0.688374,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.934679,-1.029298,0.623056,0.648722,-0.883370
4,-0.717866,-0.423979,-0.581977,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.950194,-0.868354,0.845421,1.120548,-0.655686
5,-0.552873,-0.100039,-0.431653,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.968970,-0.729501,1.075288,1.413521,-0.504317
6,-0.135459,0.163238,0.047429,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.953857,-0.628590,1.222735,1.624349,-0.475214
7,0.391154,0.366807,0.608464,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.972940,-0.261944,1.174246,1.449134,-0.074781
8,0.558032,0.511623,1.195051,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.978657,0.068702,0.653812,0.788985,0.195945
9,0.698699,0.470345,1.706389,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.490399,0.375643,0.148275,0.111366,0.449966


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      771.216000
1      855.531000
2      747.184000
3      808.342000
4      909.713000
5      991.263000
6     1167.604000
7     1277.890000
8     1261.667000
9     1152.069000
10     979.998857
11     811.194686
12     638.818457
13     584.646000
14     571.130000
15     637.028000
16     662.625000
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Ceará - Valor Cimento Portland,Distrito Federal - IDH,Distrito Federal - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - Desemprego,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado
17,2.646226,-1.170656,-0.007389,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.052055,0.751755,-0.527912,-1.765985,0.326415


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    789.947
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [15]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1027262526, 2823429873, 1934904878, 519260516, 2858157599, 1291055889, 2369630613, 2595902582, 1471573359, 3342612847, 4058568885, 3586104424, 30474497, 1313684700, 675153273, 880414443, 806957899, 1753538997, 4015794250, 3895641102, 1178645474, 3204405350, 4042386538, 968171827, 1010534880, 2550164396, 3306959012, 282885674, 28405551, 4131457467, 1044680510, 2766733795, 3759267173, 4074369866, 4177631141, 449518231, 4191767045, 972364732, 1110373508, 3594576183, 2056615522, 4234150907, 867914715, 385879869, 3551886441, 3973029498, 1475741535, 408354402, 1936929784, 4010460123, 51847490, 4203805071, 2390814577, 1848995409, 1679701160, 3686872442, 2992383511, 830870402, 3002576330, 1247828272, 2134053921, 3623147088, 2275668715, 500912131, 1671136542, 1359706500, 2763428234, 3997399601, 2409468331, 889032822, 4163528955, 3465370310, 4066845649, 656405044, 1325409329, 4079651581, 3486042285, 2667737676, 3298132046, 2509156905, 151446382, 1995123092, 603800342, 1652194580, 1196025554, 29

val_loss: 17111.849609375


Step: 85 ___________________________________________
val_loss: 1497.3118896484375


Step: 86 ___________________________________________
val_loss: 17022.189453125


Step: 87 ___________________________________________
val_loss: 2255.406005859375


Step: 88 ___________________________________________
val_loss: 1485.6837158203125


Step: 89 ___________________________________________
val_loss: 1502.5450439453125


Step: 90 ___________________________________________
val_loss: 1616.083984375


Step: 91 ___________________________________________
val_loss: 1580.2923583984375


Step: 92 ___________________________________________
val_loss: 3288.200439453125


Step: 93 ___________________________________________
val_loss: 12579.1767578125


Step: 94 ___________________________________________
val_loss: 17101.65625


Step: 95 ___________________________________________
val_loss: 1489.1685791015625


Step: 96 ___________________________________________
val_loss: 214

In [16]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 467ms/step - loss: 811646.0000 - val_loss: 395027.4062
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 762921.1250 - val_loss: 204122.6250
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 989619.5000 - val_loss: 319995.1562
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 678232.9375 - val_loss: 467418.3438
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 710599.9375 - val_loss: 55323.9258
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 544720.0625 - val_loss: 52208.3945
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 707893.6875 - val_loss: 308299.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 623262.3750 - val_loss: 209618.5781
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 486345.4688 - val_loss: 166080.1406
Epoch 10/10

1/1 [==============================] - 0s 15ms/step - loss: 123381.8203 - val_loss: 111961.7812
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 133061.9219 - val_loss: 66832.7422
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 134253.9375 - val_loss: 50322.5742
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 141694.4531 - val_loss: 56964.1914
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 146648.1250 - val_loss: 53986.4062
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 153643.6875 - val_loss: 77641.2422
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 120406.2891 - val_loss: 96913.4375
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 128583.6484 - val_loss: 76574.5938
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 144797.5781 - val_loss: 34628.1133
Epoch 84/10000
1/1 [====

1/1 [==============================] - 0s 14ms/step - loss: 56685.2930 - val_loss: 19147.6074
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 61575.6055 - val_loss: 11194.6904
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 63516.1836 - val_loss: 14825.2373
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 54619.9453 - val_loss: 16585.1973
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 52738.2852 - val_loss: 10588.6025
Epoch 154/10000
1/1 [==============================] - 0s 18ms/step - loss: 58078.9883 - val_loss: 12983.1396
Epoch 155/10000
1/1 [==============================] - 0s 18ms/step - loss: 58534.9414 - val_loss: 10350.7783
Epoch 156/10000
1/1 [==============================] - 0s 17ms/step - loss: 61404.2383 - val_loss: 6602.7554
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 56122.4805 - val_loss: 9032.6240
Epoch 158/10000
1/1 [=======

1/1 [==============================] - 0s 14ms/step - loss: 26496.0859 - val_loss: 5256.5308
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 31581.5449 - val_loss: 5866.5020
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 31539.2129 - val_loss: 5409.8774
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 35658.2852 - val_loss: 3905.1829
Epoch 228/10000
1/1 [==============================] - 0s 18ms/step - loss: 29081.7812 - val_loss: 5064.7202
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 29902.9258 - val_loss: 3713.9812
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 26641.0078 - val_loss: 4570.1470
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 32113.7051 - val_loss: 4377.1099
Epoch 232/10000
1/1 [==============================] - 0s 15ms/step - loss: 30567.8906 - val_loss: 4200.1650
Epoch 233/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 18287.2324 - val_loss: 5251.7964
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 22730.7188 - val_loss: 3561.8533
Epoch 301/10000
1/1 [==============================] - 0s 18ms/step - loss: 20164.5312 - val_loss: 3318.2332
Epoch 302/10000
1/1 [==============================] - 0s 18ms/step - loss: 23538.9277 - val_loss: 3181.1084
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 20753.0840 - val_loss: 4340.6016
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 17763.7246 - val_loss: 2292.2256
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 21190.7363 - val_loss: 2813.1074
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 18940.9102 - val_loss: 2679.2786
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 18019.9082 - val_loss: 2981.9473
Epoch 308/10000
1/1 [==============

...

1/1 [==============================] - 0s 14ms/step - loss: 8826.7578 - val_loss: 2375.8958
Epoch 1126/10000
1/1 [==============================] - 0s 13ms/step - loss: 9729.1914 - val_loss: 2501.6052
Epoch 1127/10000
1/1 [==============================] - 0s 13ms/step - loss: 8916.6963 - val_loss: 2370.2263
Epoch 1128/10000
1/1 [==============================] - 0s 13ms/step - loss: 8832.6494 - val_loss: 2359.7649
Epoch 1129/10000
1/1 [==============================] - 0s 14ms/step - loss: 8819.7637 - val_loss: 1642.2656
Epoch 1130/10000
1/1 [==============================] - 0s 14ms/step - loss: 8675.3945 - val_loss: 1636.6455
Epoch 1131/10000
1/1 [==============================] - 0s 14ms/step - loss: 10178.1191 - val_loss: 1807.6090
Epoch 1132/10000
1/1 [==============================] - 0s 14ms/step - loss: 9738.4531 - val_loss: 1537.4476
Epoch 1133/10000
1/1 [==============================] - 0s 14ms/step - loss: 8801.0684 - val_loss: 1427.7474
Epoch 1134/10000
1/1 [=============

1/1 [==============================] - 0s 14ms/step - loss: 7832.5430 - val_loss: 979.7378
Epoch 1201/10000
1/1 [==============================] - 0s 14ms/step - loss: 8128.3623 - val_loss: 875.0037
Epoch 1202/10000
1/1 [==============================] - 0s 13ms/step - loss: 8321.9697 - val_loss: 871.4402
Epoch 1203/10000
1/1 [==============================] - 0s 13ms/step - loss: 8270.5879 - val_loss: 909.0240
Epoch 1204/10000
1/1 [==============================] - 0s 14ms/step - loss: 7704.5098 - val_loss: 859.2297
Epoch 1205/10000
1/1 [==============================] - 0s 14ms/step - loss: 9541.4580 - val_loss: 1002.7527
Epoch 1206/10000
1/1 [==============================] - 0s 13ms/step - loss: 7640.3682 - val_loss: 916.0558
Epoch 1207/10000
1/1 [==============================] - 0s 13ms/step - loss: 7761.9385 - val_loss: 851.4401
Epoch 1208/10000
1/1 [==============================] - 0s 14ms/step - loss: 8179.6138 - val_loss: 881.9848
Epoch 1209/10000
1/1 [======================

1/1 [==============================] - 0s 14ms/step - loss: 8414.8398 - val_loss: 1073.1533
Epoch 1276/10000
1/1 [==============================] - 0s 14ms/step - loss: 9248.3740 - val_loss: 1223.1534
Epoch 1277/10000
1/1 [==============================] - 0s 14ms/step - loss: 8310.3936 - val_loss: 1159.6132
Epoch 1278/10000
1/1 [==============================] - 0s 14ms/step - loss: 8660.9736 - val_loss: 1227.0304
Epoch 1279/10000
1/1 [==============================] - 0s 14ms/step - loss: 8963.5947 - val_loss: 1164.2119
Epoch 1280/10000
1/1 [==============================] - 0s 14ms/step - loss: 8815.6377 - val_loss: 1476.3112
Epoch 1281/10000
1/1 [==============================] - 0s 18ms/step - loss: 9254.6641 - val_loss: 1188.6469
Epoch 1282/10000
1/1 [==============================] - 0s 18ms/step - loss: 7890.7031 - val_loss: 1149.2716
Epoch 1283/10000
1/1 [==============================] - 0s 15ms/step - loss: 8797.4424 - val_loss: 1179.3584
Epoch 1284/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 8773.2715 - val_loss: 2820.6301
Epoch 1351/10000
1/1 [==============================] - 0s 14ms/step - loss: 7393.1660 - val_loss: 2356.6780
Epoch 1352/10000
1/1 [==============================] - 0s 14ms/step - loss: 8471.7979 - val_loss: 2728.0957
Epoch 1353/10000
1/1 [==============================] - 0s 14ms/step - loss: 8774.8193 - val_loss: 2730.8899
Epoch 1354/10000
1/1 [==============================] - 0s 14ms/step - loss: 8549.6631 - val_loss: 2933.3965
Epoch 1355/10000
1/1 [==============================] - 0s 14ms/step - loss: 9375.9346 - val_loss: 3291.4258
Epoch 1356/10000
1/1 [==============================] - 0s 14ms/step - loss: 9172.2139 - val_loss: 2999.0188
Epoch 1357/10000
1/1 [==============================] - 0s 14ms/step - loss: 8253.5391 - val_loss: 3334.9915
Epoch 1358/10000
1/1 [==============================] - 0s 14ms/step - loss: 9308.2979 - val_loss: 3913.9895
Epoch 1359/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 8971.3008 - val_loss: 1620.8073
Epoch 1426/10000
1/1 [==============================] - 0s 14ms/step - loss: 9909.9424 - val_loss: 1492.2266
Epoch 1427/10000
1/1 [==============================] - 0s 14ms/step - loss: 8649.8662 - val_loss: 1926.7201
Epoch 1428/10000
1/1 [==============================] - 0s 13ms/step - loss: 8397.7754 - val_loss: 1542.4701
Epoch 1429/10000
1/1 [==============================] - 0s 14ms/step - loss: 8168.2168 - val_loss: 1593.2046
Epoch 1430/10000
1/1 [==============================] - 0s 14ms/step - loss: 8622.5771 - val_loss: 1610.5348
Epoch 1431/10000
1/1 [==============================] - 0s 14ms/step - loss: 8095.8408 - val_loss: 1738.0625
Epoch 1432/10000
1/1 [==============================] - 0s 13ms/step - loss: 8319.9141 - val_loss: 1534.4642
Epoch 1433/10000
1/1 [==============================] - 0s 14ms/step - loss: 8245.7461 - val_loss: 1558.9146
Epoch 1434/10000
1/1 [==============

In [17]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[789.947](test_target) - [[637.58856]](prediction) = 152.35843798828125


In [18]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [19]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Ceará - Valor Cimento Portland,Distrito Federal - IDH,Distrito Federal - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - Desemprego,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado
0,-1.032851,-1.752101,-1.548390,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.843956,-1.588056,-1.800731,-1.945695,-1.855558
1,-1.274753,-1.110551,-0.999274,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.032208,-1.060452,-0.159464,-1.108493,-1.070321
2,-0.838792,-0.540805,-0.571640,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.723494,-0.578996,-0.155923,-0.413979,-0.447585
3,-0.258420,-0.041800,-0.057599,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.268766,-0.137894,-0.999808,0.190053,0.079410
4,0.157908,0.387527,0.093721,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.595121,0.249146,-0.133091,0.584392,0.451052
5,0.404379,0.748241,0.307515,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.990050,0.583062,0.762865,0.829250,0.698127
6,1.027927,1.041405,0.988876,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.672165,0.825736,1.337573,1.005455,0.745631
7,1.814601,1.268083,1.786792,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.073556,1.707453,1.148579,0.859015,1.399244


train_target:


,Distrito Federal - Consumo de Cimento (t)
0,771.216
1,855.531
2,747.184
3,808.342
4,909.713
5,991.263
6,1167.604
7,1277.890


test_input:


...

,Ceará - Valor Cimento Portland,Distrito Federal - IDH,Distrito Federal - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - Desemprego,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado
17,2.646226,-1.170656,-0.007389,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.052055,0.751755,-0.527912,-1.765985,0.326415


test_target:


,Distrito Federal - Consumo de Cimento (t)
17,789.947


1/1 [==============================] - 0s 36ms/step
Error: 152.35843798828125




[927.3223266601562,
 1245.433349609375,
 1159.1451416015625,
 1041.6602783203125,
 1081.97021484375,
 956.9374389648438,
 792.5532836914062,
 897.5900268554688,
 762.4633178710938,
 637.5885620117188]

In [20]:
display(targets)
display(predictions)

[1261.667,
 1152.069,
 979.9988571428572,
 811.1946857142857,
 638.8184571428571,
 584.646,
 571.13,
 637.028,
 662.625,
 789.947]

[927.3223266601562,
 1245.433349609375,
 1159.1451416015625,
 1041.6602783203125,
 1081.97021484375,
 956.9374389648438,
 792.5532836914062,
 897.5900268554688,
 762.4633178710938,
 637.5885620117188]

In [21]:
mae = mean_absolute_error(predictions, targets)
mae

238.69461630859377

In [22]:
porcentage = mae/np.mean(targets)
porcentage

0.29508092138109615